<H1>Construcción de la red </H1>

In [1]:
from typing import Dict

import numpy as np

from influenceHeuristics import identify_nodes, build_mentions_matrix
from processData import preprocess_dataframe, build_users_tweet_text, create_link_processor
import networkx as nx
import pandas as pd


df = pd.read_csv('ReformaPensional.csv')
df = preprocess_dataframe(df)
users = identify_nodes(df)
user_index: Dict[str, int] = {user: idx for idx, user in enumerate(users)}
index_user: Dict[int, str] = {index: user for user, index in user_index.items()}
users_tweet_text = build_users_tweet_text(df, user_index)
get_links_matrix = create_link_processor(index_user)

C:\Users\jujov\TG_code\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\jujov\TG_code\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
mentions_matrix, mentions_matrix_date,  mentions_matrix_nonNorm= build_mentions_matrix(df, user_index)
mentions_G = nx.from_numpy_array(mentions_matrix, create_using=nx.DiGraph(directed=True))
nx.relabel_nodes(mentions_G, index_user, copy=False)
nx.write_gexf(mentions_G, "mentions-1.gexf")

In [4]:
from influenceHeuristics import build_global_influence_matrix

global_influence_matrix, global_influence = build_global_influence_matrix(df, user_index, mentions_matrix_nonNorm)
global_influence_G = nx.from_numpy_array(global_influence_matrix, create_using=nx.DiGraph(directed=True))
nx.relabel_nodes(global_influence_G, index_user, copy=False)
nx.write_gexf(global_influence_G, "popularity-1.gexf")

In [6]:
from influenceHeuristics import build_local_influence_matrix

local_influence_matrix = build_local_influence_matrix(df, user_index, global_influence, mentions_matrix_nonNorm)
local_influence_G = nx.from_numpy_array(local_influence_matrix, create_using=nx.DiGraph(directed=True))
nx.relabel_nodes(local_influence_G, index_user, copy=False)
nx.write_gexf(local_influence_G, "relativePopularity-1.gexf")

In [3]:
from processData import calculate_stance
from influenceHeuristics import build_agreement_clique_matrix
topic = "La reforma pensional en Colombia"
topic_context = (
        "El gobierno del presidente Gustavo Petro presentó un proyecto de ley para reformar el sistema pensional, "
        "el cual fue aprobado con el apoyo de algunos sectores y cuestionado por otros.")
prompt = (
        f"Eres un analista experto e imparcial en {topic}. Tu tarea es medir la postura de un usuario frente a este tema en un rango continuo entre 0 y 1, donde: "
        f"0 = Férreamente en contra, 1 = Reciamente a favor, 0.5 = Neutral, "
        f"0.25 = Oposición moderada (reconoce o no está enfáticamente indispuesto a reconocer aspectos positivos), "
        f"0.75 = Apoyo moderado (reconoce o está predispuesto a reconocer objeciones)."
        f"Contexto: \"{topic_context}\" "
        "Entrada: Opiniones del usuario desde una red social"
        "Instrucciones: "
        "1. Analiza la postura general en cada opinión"
        "2. Calcula la proporción de frases a favor vs. en contra del tema"
        "3. Emociones explícitas directas hacia el tema refuerzan la postura (más emoción positiva = postura cercana a 1; negativa = cercana a 0). "
        "4. Pondera adjetivos y adverbios utilizados para expresar alguna postura frente al tema"
        "5. Conectores de oposición indican postura moderada. Ej: 'aunque no es perfecta, la reforma pensional es un paso positivo' o 'a pesar de las críticas, tiene aspectos rescatables'"
        "6. Los adverbios de cantidad indican posturas extremas, y los de duda, posturas moderadas"
        f"Salida esperada: Un número decimal entre 0 y 1, con dos decimales, como 0.21, 0.75 o 0.03, representando la postura general del usuario frente a {topic}.")

stances = calculate_stance(users_tweet_text, users, prompt)
agreement_clique_matrix= build_agreement_clique_matrix(users_tweet_text, stances, index_user, 0.1)

len(agreement_clique_matrix[agreement_clique_matrix!=0]), len(agreement_clique_matrix[agreement_clique_matrix==-1])

(2288730, 1453955)

In [3]:
agreement_clique_G = nx.from_numpy_array(agreement_clique_matrix)
nx.relabel_nodes(agreement_clique_G, index_user, copy=False)
nx.write_gexf(agreement_clique_G, "agreementClique-1.gexf")

In [4]:
from influenceHeuristics import build_agreement_matrix

agreement_matrix= build_agreement_matrix(mentions_matrix, stances, index_user, 0.2)

print(len(agreement_matrix[agreement_matrix!=0]),len(agreement_matrix[agreement_matrix==-1]))

1064 0


In [5]:
agreement_G = nx.from_numpy_array(agreement_matrix)
nx.relabel_nodes(agreement_G, {i: f"{user} ({stances[user]})" for i, user in index_user.items()}, copy=False)
nx.set_node_attributes(agreement_G, {f"{user} ({stances[user]})":  stances[user] for i, user in index_user.items()}, "stance")
nx.write_gexf(agreement_G, "agreement-3.gexf")

In [6]:
from influenceHeuristics import build_affinities_matrix

affinities_matrix = build_affinities_matrix(users_tweet_text, stances, index_user, mentions_matrix_nonNorm)

print(len(affinities_matrix[affinities_matrix!=0]))

i=0, 1732225930.951495
i=1, 1732225947.69688
i=2, 1732225947.7593954
i=3, 1732225949.5143664
i=4, 1732225949.5772173
i=5, 1732225949.6719751
i=6, 1732225954.134158
i=8, 1732225954.182184
i=10, 1732225954.2595096
i=12, 1732225954.3224556
i=13, 1732225954.3872216
i=14, 1732225954.4704278
i=15, 1732225954.533962
i=16, 1732225954.600225
i=17, 1732225954.6468709
i=18, 1732225954.6629252
i=20, 1732225954.7265558
i=21, 1732225954.8058681
i=22, 1732225954.8861847
i=23, 1732225955.2342608
i=24, 1732225955.2498903
i=25, 1732225955.2655134
i=26, 1732225955.3464687
i=27, 1732225955.4014082
i=28, 1732225955.4742138
i=29, 1732225955.6013625
i=30, 1732225958.4450893
i=31, 1732225958.4607325
i=32, 1732225958.524237
i=33, 1732225958.5554857
i=34, 1732225958.5867448
i=35, 1732225958.6521568
i=37, 1732225958.66783
i=38, 1732225958.7468405
i=39, 1732225958.811719
i=40, 1732225958.8778827
i=41, 1732225958.9570231
i=42, 1732225959.0204406
i=43, 1732225959.0870092
i=44, 1732225959.1664772
i=45, 1732225959.18

In [8]:
affinities_G = nx.from_numpy_array(affinities_matrix, create_using=nx.DiGraph(directed=True))
nx.relabel_nodes(affinities_G, index_user, copy=False)
nx.write_gexf(affinities_G, "affinities-1.gexf")

In [14]:
import numpy as np
x=np.array([[1,9],[2, 2]])
print(x, type(x[0]))

[[1 9]
 [2 2]] <class 'numpy.ndarray'>


In [20]:
data = [1,2,3,4,5,6]
def group_in_batches(data, size):
        return [data[i:i + size] for i in range(0, len(data), size)]
group_in_batches(data, 3)

[[1, 2, 3], [4, 5, 6]]

In [24]:
float("8.64")

8.64

In [17]:
from stanceHeuristics import Stance
from openai import OpenAI
from dotenv import dotenv_values

openIAKey = dotenv_values(".env")["OPENAI_API_KEY"]
client = OpenAI(api_key=openIAKey)
users_opinions = []
completion = client.beta.chat.completions.parse(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": "{'@petrogustavo Ahora son los ancianos la CORTINA DE HUMO, usándolos para despertar sentimientos de pesar en los colombianos. No sé trata de ellos señor, usted sólo quiere esconder todos los vicios de la Reforma Pensional y tratar de evitar que se caiga en la Corte Constitucional.'}"},
                    {"role": "user", "content": "{'Se oponen a la reforma pensional que le ofrece dignidad a los viejitos del país, pero defienden a Doña Segunda por incumplir con la factura electrónica de su negocio, ante la DIAN. ¡Yo no los entiendos, bobos hijueputas! '}"}
                ],
                response_format=Stance,
            )

In [18]:
for choice in completion.choices:
    print(choice.message.content)

{"value":0.03}
